# Setup (just run)

In [2]:
# Colab-specific setup

# !git clone https://github.com/AISC-Steering-LLMs/Steering-LLMs
# !pwd
# repo_path = '/content/repository/'


In [3]:
# Imports
import pandas as pd
import main
from omegaconf import DictConfig, OmegaConf
import yaml
from hydra import initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.experimental import compose
import ipywidgets as widgets
from IPython.display import display

# For refactored code
# Need to tidy this up and remove duplicates

from data_handler import DataHandler
from data_analyser import DataAnalyzer
from model_handler import ModelHandler

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration

# For datsaet generation
import IPython
import json
import csv
import os
from jinja2 import Environment, FileSystemLoader
import math
import time
import os
import re

import yaml
from ipywidgets import widgets, VBox, Button, Checkbox, Text, IntText, FloatText, SelectMultiple, Label

from openai import OpenAI
client = OpenAI()

# Data generation

Skip to experiment runs section if you don't want to generate a new dataset

## Inputs

In [5]:
# Note for gpt-4-0125-preview, the maximum number of tokens is 4096
# including the prompt and the response
# Based on Eleni's latest prompts
# Assuming an input prompt of 200 words = 250 tokens
# And assuming 50 tokens per generated prompt example we want in the response
# We can expect to generate a maximum of (4096-250)/50 = 3846/50 = 76.92
# So something like 75 generated examples per prompt is the max we can ask for in one go.
# 250 prompt tokens = 200 * 0.01/1000 = $0.0025
# 3846 completion tokens = 3846 * 0.03/1000 = $0.11538
# So the total cost is $0.11788 per 75 prompts.
# If we wanted to generate 1000 prompts, it would cost $1.5718
# Please check your prompts work with ChatGPT before generating a large dataset using the API

model = "gpt-4-0125-preview"
prompt_structure_dir = "pairs_v1"
template_file = "template_multi.j2"
prompt_context_file = "honesty.json"
num_examples_per_prompt = "5" # Must be a whole number inside quote marks. Max is 75.
total_num_examples = 10

## Define paths

In [6]:
# Constants
SRC_PATH = "../data/inputs"
DATASET_BUILDER_DIR_PATH = os.path.join(SRC_PATH, "prompts", prompt_structure_dir)

# Number of interations of the prompt to generate the entire dataset
num_iterations = math.ceil(total_num_examples/int(num_examples_per_prompt))

# Input directories and files
template_file_path = os.path.join(DATASET_BUILDER_DIR_PATH, "templates", template_file)
prompt_context, _ = os.path.splitext(prompt_context_file)
prompt_context_file_path = os.path.join(DATASET_BUILDER_DIR_PATH, "contexts", prompt_context+".json")

# Output directories and files
dataset_generator_prompt_file_path = os.path.join(DATASET_BUILDER_DIR_PATH, "dataset_generator_prompts", prompt_context+"_prompt.txt")
generated_dataset_dir = os.path.join(DATASET_BUILDER_DIR_PATH, "generated_datasets")
generated_dataset_file_path = os.path.join(generated_dataset_dir, prompt_context+"_dataset")
log_file_path = os.path.join(DATASET_BUILDER_DIR_PATH, "logs", prompt_context+"_log")
combined_dataset_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset.csv")

## Helper functions

In [7]:
# Forming the prompt from the template and template material
def render_template_with_data(template_file_path,
                              prompt_context_file_path,
                              dataset_generator_prompt_file_path,
                              num_examples_per_prompt):

    # Set up the environment with the path to the directory containing the template
    env = Environment(loader=FileSystemLoader(os.path.dirname(template_file_path)))

    # Now, get_template should be called with the filename only, not the path
    template = env.get_template(os.path.basename(template_file_path))
    
    # Load the prompt context
    with open(prompt_context_file_path, 'r') as file:
        prompt_construction_options = json.load(file)

    # Update the prompt context example to replace the list with the combined string
    example_text = "\n".join(prompt_construction_options["example"])
    prompt_construction_options["example"] = example_text
    prompt_construction_options["num_examples"] = num_examples_per_prompt

    # Render the template with the prompt_construction_options
    prompt_to_generate_dataset = template.render(prompt_construction_options)

    # Save the prompt to a file
    with open(dataset_generator_prompt_file_path, 'w') as file:
        file.write(prompt_to_generate_dataset)

    # Remove newlines from the prompt and replace with spaces
    prompt_to_generate_dataset = prompt_to_generate_dataset.replace('\n', ' ')

    # Save the prompt to a file
    with open(dataset_generator_prompt_file_path, 'w') as file:
        file.write(prompt_to_generate_dataset)

    return prompt_to_generate_dataset



# Generate the dataset by calling the OpenAI API
def generate_dataset_from_prompt(prompt,
                                 generated_dataset_file_path,
                                 model,
                                 log_file_path,
                                 i):
    completion = client.chat.completions.create(
            **{
                "model": model,
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            }
        )
    
    completion_words = completion.choices[0].message.content.strip()

    # cleaned_completion = completion.choices[0].message.content.strip()[3:-3]
    print(" ")
    print(completion_words)
    print(" ")

    # Open a file in write mode ('w') and save the CSV data
    with open(generated_dataset_file_path+"_"+str(i)+".txt", 'w', newline='', encoding='utf-8') as file:
        file.write(completion_words)

    num_words_in_prompt = count_words_in_string(prompt)
    num_words_in_completion = count_words_in_string(completion_words)
    total_words = num_words_in_prompt + num_words_in_completion

    num_tokens_in_prompt = completion.usage.prompt_tokens
    num_tokens_in_completion = completion.usage.completion_tokens
    total_tokens = num_tokens_in_prompt + num_tokens_in_completion

    prompt_cost = num_tokens_in_prompt*0.01/1000
    completion_cost = num_tokens_in_completion*0.03/1000
    total_cost = prompt_cost + completion_cost
    
    tokens_per_prompt_word = num_words_in_prompt/num_tokens_in_prompt
    tokens_per_completion_word = num_words_in_completion/num_tokens_in_completion

    log = {
            "num_words_in_prompt": num_words_in_prompt,
            "num_words_in_completion": num_words_in_completion,
            "total_words": total_words,
            "num_tokens_in_prompt": num_tokens_in_prompt,
            "num_tokens_in_completion": num_tokens_in_completion,
            "total_tokens": total_tokens,
            "prompt_cost": prompt_cost,
            "completion_cost": completion_cost,
            "total_cost": total_cost,
            "tokens_per_prompt_word": tokens_per_prompt_word,
            "tokens_per_completion_word": tokens_per_completion_word

    }

    for k, v in log.items():
        print(k, v)
    print(" ")

    with open(log_file_path+"_"+str(i)+".txt", 'w') as file:
        file.write(json.dumps(log, indent=4))

def count_words_in_string(input_string):
    words = input_string.split()
    return len(words)

In [8]:
start_time = time.time()

# Generate the prompt
prompt = render_template_with_data(template_file_path,
                                   prompt_context_file_path,
                                   dataset_generator_prompt_file_path,
                                   num_examples_per_prompt,
                                   )

# Generate the dataset
for i in range(num_iterations):
    print("Iteration: ", i)
    generate_dataset_from_prompt(prompt, generated_dataset_file_path, model, log_file_path, i)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

Iteration:  0


 
"How is honesty important in maintaining friendships? Select from the following options: A. It builds trust and strengthens the relationship. B. It’s not important; you can make up things as you go. The answer is A because",  
"How is honesty important in maintaining friendships? Select from the following options: A. It builds trust and strengthens the relationship. B. It’s not important; you can make up things as you go. The answer is B because",  
"When is being honest most crucial? Select from the following options: A. In situations where trust could be broken. B. Only when you are likely to get caught lying. The answer is A because",  
"When is being honest most crucial? Select from the following options: A. In situations where trust could be broken. B. Only when you are likely to get caught lying. The answer is B because",  
"What role does honesty play in a professional setting? Select from the following options: A. It ensures transparency and builds trust among colleagues. B. 

## Combine datasets

In [9]:
# Get a list of all the files you want to process
# Makes sure they all have the same prompt context
# eg won;t mix up honest with justice etc
files = [os.path.join(generated_dataset_dir, f) for f in os.listdir(generated_dataset_dir) if f.endswith('.txt') and prompt_context in f]

# Define the regular expression pattern
# Get lines that start with a quote,
# then have any number of characters,
# then end with a quote and possibly comma
# We're trying to find all valid CSV lines
pattern = r'^\".*\",?[\r\n]*'

# Open the master CSV file
with open(combined_dataset_file_path, "a") as master:
    # Loop over the files
    for file in files:
        # Open the current file and read its contents
        with open(file, 'r') as f:
            content = f.read()

        # Use the re.findall function to find all matches in the content
        matches = re.findall(pattern, content, re.MULTILINE)        

        # Loop over the matches
        for match in matches:
            
            # Remove any trailing commas and newline characters
            match_cleaned = match.rstrip(',\r\n')
            
            # Append the match to the master CSV file
            master.write(match_cleaned + '\n')

## Add optional columns for classification

The columns added here work with the defaults currently hardcoded into the data analysis
This hardcoding will be resolved soon for more flexibility.

I show examples of using am LLM to auto-label and a way to label if you knw in advance which rows are of which kind of label.

### Example of labelling using an LLM

In [10]:
def ask_openai(prompt):
    completion = client.chat.completions.create(
                **{
                    "model": model,
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ]
                }
            )
    return completion.choices[0].message.content.strip()

input_file_path = combined_dataset_file_path
output_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_ethical_area.csv")


with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Add a header
    writer.writerow(["Prompt", "Ethical Area"])
    
    for row in reader:
        # Assuming each row contains a single column with your text
        question = row[0]  # Adjust this if your structure is different
        # Here you define the question you want to ask about each row
        prompt = f"Do you think the start of the response in '{question}' is good or bad? Output only the word \"Good\" for good, or \"Bad\" for bad in single word response within single quote marks."
        response = ask_openai(prompt)
        # Add the OpenAI response to the row
        row.append(response)
        writer.writerow(row)

### Example of labelling programatically without LLM

In [11]:
# Note we might not need to query the API for kind of lebelling
# Eg if we know the questions always go good, bad, good, bad, good etc

input_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_ethical_area.csv")
output_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_fully_labelled.csv")

with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # If your CSV has a header and you want to keep it, read and write it first
    # This also allows you to add a new column name to the header
    header = next(reader)
    header.append("Positive")  # Add your new column name here
    writer.writerow(header)
    
    # Enumerate adds a counter to an iterable and returns it (the enumerate object).
    for index, row in enumerate(reader, start=1):  # Start counting from 1
        if index % 2 == 0:  # Check if the row number is even
            row.append(0)
        else:
            row.append(1)
        writer.writerow(row)